<a href="https://colab.research.google.com/github/bartronicus/lstm-test/blob/master/TCN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from multiprocessing import Pool
import pandas as pd
import numpy as np
from google.colab import files, drive
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import datetime
from tqdm import tqdm

In [2]:
import sys
!git clone https://github.com/locuslab/TCN.git
p = '/content/TCN/TCN/'
if p not in sys.path:
  sys.path.insert(0, p)
print(sys.path)
from tcn import TemporalConvNet

Cloning into 'TCN'...
remote: Enumerating objects: 143, done.
remote: Total 143 (delta 0), reused 0 (delta 0), pack-reused 143
Receiving objects: 100% (143/143), 16.22 MiB | 21.68 MiB/s, done.
Resolving deltas: 100% (61/61), done.
['/content/TCN/TCN/', '', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [4]:
drive.mount('/content/gdrive')
import os

raw = 0
for i,f in enumerate(os.listdir('/content/gdrive/MyDrive/klines')):
  # if i > 12: 
  #   continue
  path = '/content/gdrive/MyDrive/klines/' + f
  csv = pd.read_csv(path)
  if type(raw) == int:
    raw = csv
  else:
    raw = raw.append(csv)

# raw = pd.read_csv('/content/gdrive/MyDrive/klines/BTCUSDT-klines.csv')
# raw = raw.append(pd.read_csv('/content/gdrive/MyDrive/klines/ETHUSDT-klines.csv'))
# raw = raw.append(pd.read_csv('/content/gdrive/MyDrive/klines/DOGEUSDT-klines.csv'))
# btc.info()
# raw.info()
# raw.info()
raw.sort_values(by=['symbol', 'openTime'],inplace=True)

Mounted at /content/gdrive


In [ ]:
raw.head()
# test = raw
# # test['closeTime'] = pd.to_datetime(test['closeTime'], unit='ms')
# test.head()
# # print(test['closeTime'].max())
# test['closeTime'].min()
# test.info()
raw.info()

In [5]:
cols = ['open', 'high', 'low', 'close','volume','quoteVolume','numberOfTrades','takerBuyBaseVolume','takerBuyQuoteVolume']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
from sklearn.preprocessing import MinMaxScaler
data = raw.dropna()
minutes = 60
data = raw.dropna()
data.index = pd.to_datetime(data['closeTime'], unit="ms")

data['y'] = data['close'].shift(-minutes)
# when we shift the closing price data from some coin may get assigned to another coin
# just drop wherever this happens
data['symbol_check'] = data['symbol'].shift(-minutes)
data['close_time_test'] =  data['closeTime'].shift(-minutes)
# check that the y value is actually from [minutes] in the future
data['diff'] = data['close_time_test'] - data['closeTime']
data = data[data['diff'] == 60000*minutes]

data.reset_index(drop = True, inplace=True)
data.dropna(inplace=True)

data = data[data['symbol'] == data['symbol_check']]

# # tolerate a 2 minute gap
data['diff'] = data['closeTime'].shift(-1) - data['closeTime']
data = data[data['diff'] < (60000 * 2)]
data.dropna(inplace=True)




In [7]:
pd.options.mode.chained_assignment = None  # default='warn'
series_len = 1000

data['series'] = (data['closeTime'].shift(-1) - data['closeTime']) > 60000 * 2
data['series'] = data['series'] + (data['symbol'] != data['symbol_check'])
data['series'] = data['series'] * data.index
data['series'] = data.groupby(data['series']).ngroup()
data['series'] = data['series'].replace(0,np.NaN)
data['series'][0] = 0
data['series'].unique()
data['series'] = data['series'].fillna(method='ffill')
data['series'] = data['series'].shift(1)
data = data.dropna()
data['series_index'] = data.groupby('series').cumcount()

# DROP SERIES BELOW MINIMUM LENGTH 
series_len_df = data.groupby(data['series']).size().reset_index(name='series_len')
data = data.merge(series_len_df, on='series', how='left')
data = data[data['series_len'] >= series_len]
data = data.reset_index()


/usr/local/lib/python3.7/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [8]:
losses = []

df = pd.DataFrame(columns = data.columns)

for series in data.series.unique():
  series = data[data['series'] == series]
  scaler = MinMaxScaler()
  scaler.fit(data[[*cols,'y']])
  series[[*cols,'y']] = scaler.fit_transform(series[[*cols,'y']])
  df = df.append(series)  

  # benchmark
  # what if we just used the last price to predict the next?
  loss_fn = nn.MSELoss()
  close = torch.tensor(series['close'].to_numpy())
  y = torch.tensor(series['y'].to_numpy())
  loss = loss_fn(close,y)
  losses.append(loss.item())

In [9]:
del data
print(np.asarray(losses).sum() / len(losses))
print('\n')
losses

0.00020580091812136676




[0.0012901556831559904,
 0.00018105799022889252,
 0.000530716150873572,
 7.692526895257364e-05,
 8.163464005221232e-05,
 0.0001639402783458955,
 5.351250869534212e-05,
 2.4835866963275576e-05,
 0.00013283638288111635,
 3.603844284608843e-05,
 0.00018509666633841808,
 4.990471745535988e-05,
 6.615631225537079e-05,
 2.8683432180140504e-05,
 0.0001582282926258355,
 9.069086067724878e-05,
 0.00010205045778532901,
 4.6304711270886524e-05,
 4.014545304909859e-05,
 0.00015473836386807155,
 0.0003081559719865387,
 5.191601014388456e-06,
 2.1217897427267217e-06,
 0.00034641535168244964,
 4.9037176146974195e-05,
 0.00022225596274657377,
 5.7813290565120545e-05,
 9.290684708018316e-05,
 0.00011738237246334708,
 0.00016464194319644316,
 8.274838904520969e-05,
 0.0016433062037130627]

In [10]:
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], num_channels[-1])
        self.linear_out = nn.Linear(num_channels[-1], output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        output = self.tcn(x.transpose(1, 2)).transpose(1, 2)
        output = self.linear(output)
        output = self.relu(output)
        # output = torch.squeeze(output)
        output = self.linear_out(output)
        output = torch.sum(output,1)
        output = torch.squeeze(output)
        # print(output,output.shape)
        return output

        # output = self.linear(output).double()
        # output = self.sig(output)
        # output = torch.sum(output)


In [11]:
import random
from torch.utils.data import Dataset, DataLoader, IterableDataset
from itertools import chain, islice, cycle, filterfalse

batch_size = 64

class TimeSeries(IterableDataset):
  
  def __init__(self, data_list):
    self.data_list = data_list
  
  def process_data(self, df):
    valid_rows = df[df['series_index'] >= series_len]
    data_cols = df[cols]
    for i in valid_rows.index:
      x = data_cols.iloc[i-series_len:i].to_numpy()
      yield {
          "x": x,
          "y": df['y'].at[i]
      }
  
  def get_stream(self, data_list):
    return chain.from_iterable(map(self.process_data,cycle(data_list)))
  
  def __iter__(self):
    return self.get_stream(self.data_list)

def get_df_series(series_int):
  d = df[df['series'] == series_int]
  return d.reset_index()

series_list = list(map(get_df_series, df.series.unique()))
random.Random(0).shuffle(series_list)

train_len = int(len(series_list)*.8)
test_len = int(len(series_list)*.2)

train_data = TimeSeries(series_list[:train_len])
test_data = TimeSeries(series_list[train_len:])


train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=test_len)


In [12]:
def evaluate():
  model.eval()  
  eval_loss = 0
  with torch.no_grad():
    for data in islice(test_loader,1):
      x = data['x'].to(device)
      y = data['y'].to(device)

      optimizer.zero_grad()
      output = model(x)
      loss = loss_fn(y, output)
      eval_loss = loss.item()
      writer.add_scalar("Eval" + ' loss', eval_loss, ep)
      # eval_loss = total_loss / count
      print("Eval" + " loss: {:.5f}".format(eval_loss))
      return eval_loss


In [13]:

def train(ep):
  valid_rows = int(len(df[df['series_index'] >= series_len])/batch_size)
  gen = tqdm(islice(train_loader, valid_rows),position=0,total=valid_rows)
  model.train()
  total_loss = 0
  count = 0
  
  for batch in gen:
    x = batch['x'].to(device)
    if x.shape[0] != batch_size:
      print(x.shape)
    y = batch['y'].to(device)

    optimizer.zero_grad()
    output = model(x)
    loss = loss_fn(y, output)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    count += 1

  if count > 0:
    curr_loss = total_loss / count
    writer.add_scalar('training loss', curr_loss, ep)
    print("\nEpoch ",ep,"| lr |", params["lr"], "loss", curr_loss,"\n")




*   adding a channel from 6x6+1 to 7x6+1 was good
*   improved by jumping up to 60,40,40,30,20,10,1 
      somehow, 50,40... and 60,50... were sign. worse



In [ ]:
title = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
writer = SummaryWriter('/runs/' + title)
best_tloss = 1e8
model_name = "btc.pt"
epochs = 100
torch.manual_seed(0)
random.seed(0)

params = {
  # 'num_channels': [60,40,30,20,10,4,2,1],
  # 'num_channels': [10,20,30,40,60],
  'num_channels':[1,2,3,4,6,12,24],
  'kernel_size': 3,
  'dropout': 0.75,
  'batch_size': 64,
  'lr': 1e-3
}

model = TCN(len(cols), 1, params['num_channels'], params['kernel_size'], dropout = params['dropout'])
model = model.to(dtype=torch.float64)
model = model.to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])

for ep in range(1, epochs+1):
    print('\n\nSTARTING EPOCH',ep,'\n\n')
    train(ep)
    tloss = evaluate()
    if tloss < best_tloss:
        with open(model_name, "wb") as f:
            torch.save(model, f)
            print("Saved model!")
        best_tloss = tloss
    # if ep > 10 and tloss > best_tloss: 
    #   params['lr'] /= 10
    #   for param_group in optimizer.param_groups:
    #     param_group['lr'] = params['lr']


writer.close()

In [ ]:
# %load_ext tensorboard
%tensorboard --logdir /runs

In [ ]:
from tempfile import TemporaryFile
outfile = TemporaryFile()

p = '/content/gdrive/MyDrive/binance_time_series.csv'


# valid_rows = df[df['series_index'] >= series_len]

# with open(p, 'w') as csvfile:
#   for i in tqdm(valid_rows.index, position=0):
#     x = df[cols].iloc[i-series_len:i].to_numpy()
#     x = x.ravel()
#     y = df.at[i,'y']
        
#     writ = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
#     writ.writerow([*x,y]) 



In [42]:
import h5py
import numpy as np

valid_rows = df[df['series_index'] >= series_len]
# p = '/content/gdrive/MyDrive/binance_time_series.hdf5'
# f = h5py.File(p,  "w")


# dset = f.create_dataset('dataset', (len(valid_rows),2))

# valid_rows = df[df['series_index'] >= series_len]
# x_set = f.create_dataset('xset', (len(valid_rows),series_len,len(cols)), dtype=np.float64)
# y_set = f.create_dataset('yset', (len(valid_rows),), dtype=np.float64)

m = 0

def enumerate2(xs, start=0, step=1):
    for x in xs:
        yield (start, x)
        start += step

for i,s in tqdm(enumerate(df.series.unique()),position=0):
  print('dis is a new series',i,'/',len(df.series.unique()))
  d = df[df['series'] == s]    
  d.reset_index(inplace=True)
  data_cols = d[cols]

  for j in tqdm(d.index,position=0):
    l = j + series_len
    x = data_cols.iloc[j:l].to_numpy()

    if x.sum() == 0:
      print('IT\'S BROKEN HERE WTF',i,j)
      continue

    if x.shape[0] < series_len:
      continue

    try: 
      y = d['y'][l]
      x_set[m] = x
      y_set[m] = y

      m += 1
    except:
      continue


  1%|          | 162/24797 [00:00<00:15, 1619.38it/s]

dis is a new series 0 / 32


  0%|          | 142/44419 [00:00<00:31, 1415.13it/s]

dis is a new series 1 / 32


  0%|          | 141/44409 [00:00<00:31, 1403.90it/s]

...

  0%|          | 127/44453 [00:00<00:35, 1262.72it/s]

dis is a new series 30 / 32


  1%|          | 122/19648 [00:00<00:16, 1203.78it/s]

dis is a new series 31 / 32


100%|██████████| 19648/19648 [00:14<00:00, 1322.99it/s]
32it [20:03, 37.59s/it]


In [40]:
x_set[0]

array([[0.07360252, 0.05635311, 0.07246269, ..., 0.01220069, 0.01358676,
        0.00971611],
       [0.0766594 , 0.05932382, 0.077     , ..., 0.02042588, 0.02728824,
        0.01957959],
       [0.07757943, 0.0565793 , 0.07525373, ..., 0.01325169, 0.01535505,
        0.01099257],
       ...,
       [0.09394708, 0.07482583, 0.09447761, ..., 0.00927618, 0.01367359,
        0.00993443],
       [0.09446646, 0.07390596, 0.09331343, ..., 0.00840797, 0.01048039,
        0.00761041],
       [0.09287866, 0.07560998, 0.09352239, ..., 0.01325169, 0.04768719,
        0.03465572]])